In [1]:
from lionagi import Branch, iModel, BaseModel

In [2]:
class Source(BaseModel):
    title: str
    url: str


class ResearchReport(BaseModel):
    title: str | None = None
    content: str | None = None
    source: list[Source] | None = None

In [3]:
exa = iModel(
    provider="exa",
    endpoint="search",
    queue_capacity=5,
    capacity_refresh_time=1,
    invoke_with_endpoint=False,
)

sonnet = iModel(
    provider="openrouter",
    model="anthropic/claude-3.5-sonnet",
    max_tokens=8000,  # required for anthropic models
    invoke_with_endpoint=False,
    temperature=0.2,
)

In [4]:
from lionagi.service.providers.types import ExaSearchRequest

system_message = """
You are a specialized research assistant. Your task is to thoroughly research the user's query using your available tools
"""

branch = Branch(system=system_message, chat_model=sonnet)
branch.connect(
    name="exa_search",
    imodel=exa,
    request_options=ExaSearchRequest,
    description="Search the Exa database for relevant information",
)

In [5]:
user_msg = """I want a research paper on latest trend using AI to predict the stock market"""

In [6]:
result = await branch.ReAct(
    instruct={
        "instruction": user_msg,
    },
    interpret=True,
    response_format=ResearchReport,
    extension_allowed=True,
    max_extensions=3,
)

In [7]:
from IPython.display import Markdown

msgs = f"Title: {result.title}\n\nContent: {result.content}\n\n"
for i in result.source:
    msgs += f"Source: [{i.title}]({i.url})\n\n"

Markdown(msgs)

Title: Recent Developments in AI Applications for Stock Market Prediction (2020-2023): A Comprehensive Review

Content: Abstract:
This research paper analyzes recent developments in artificial intelligence applications for stock market prediction between 2020-2023, focusing on machine learning models, implementation approaches, accuracy metrics, and associated challenges. The review synthesizes findings from peer-reviewed literature to provide insights into the current state and future directions of AI-driven stock market prediction.

1. Introduction
Stock market prediction remains a critical area of interest for investors and researchers alike. Recent advances in artificial intelligence, particularly deep learning architectures, have revolutionized prediction approaches and accuracy.

2. Current Machine Learning Models and Algorithms
Recent research demonstrates a clear shift toward deep learning architectures, with several key models emerging as leaders:

2.1 Deep Learning Architectures
- Long Short-Term Memory (LSTM) networks
- Convolutional Neural Networks (CNN)
- Hybrid models like HCRNN (Hybrid Convolutional Recurrent Neural Network)
- Deep-ConvLSTM combining convolutional and LSTM layers

2.2 Model Innovations
Researchers have developed hybrid approaches combining multiple architectures to leverage their respective strengths. The HCRNN model, for instance, uses CNN for local fluctuation features while LSTM captures long-term dependencies.

3. Real-World Implementation Examples
3.1 Data Sources and Features
- Technical indicators and historical price data
- Integration of news-based prediction systems
- Advanced feature engineering techniques

3.2 Market Applications
- Successfully implemented across multiple markets including NYSE, NSE, and Chinese markets
- News-based systems showing 3-4.5% improvement over traditional methods

4. Accuracy Rates and Limitations
4.1 Performance Metrics
- LSTM-based models: 90-92% accuracy in short-term predictions
- HCRNN model: 278.46% better returns than market baseline
- Deep-ConvLSTM: MSE of 7.2487 and RMSE of 2.6923

4.2 Limitations
- High computational requirements and infrastructure needs
- Market volatility impact on prediction accuracy
- Variable performance across different market conditions
- Reduced reliability in long-term predictions

5. Comparison with Traditional Methods
5.1 Advantages
- Superior pattern recognition in complex market data
- Better handling of non-linear relationships
- Improved adaptation to market changes

5.2 Disadvantages
- Higher implementation costs
- Greater complexity in model interpretation
- Increased data requirements

6. Ethical Considerations and Regulatory Challenges
6.1 Ethical Issues
- Model transparency and interpretability concerns
- Potential for market manipulation
- Data privacy and security considerations

6.2 Regulatory Challenges
- Varying compliance requirements across jurisdictions
- Need for standardized model validation frameworks
- Balance between innovation and market stability

7. Conclusion
AI applications in stock market prediction have shown significant advancement during 2020-2023, with deep learning models demonstrating superior performance compared to traditional methods. However, challenges remain in terms of computational requirements, model transparency, and regulatory compliance. Future research should focus on addressing these limitations while maintaining prediction accuracy.

8. Future Directions
- Development of more efficient computational methods
- Integration of multiple data sources
- Enhanced model interpretability
- Standardized regulatory frameworks

Source: [Important Trading Point Prediction Using a Hybrid Convolutional Recurrent Neural Network](https://www.mdpi.com/2076-3417/11/9/3984)

Source: [Stock Market Prediction Using Optimized Deep-ConvLSTM Model](https://www.liebertpub.com/doi/10.1089/big.2018.0143)

Source: [News-based Machine Learning and Deep Learning Methods for Stock Prediction](https://iopscience.iop.org/article/10.1088/1742-6596/1642/1/012014)

Source: [Short-term stock market price trend prediction using a comprehensive deep learning system](https://journalofbigdata.springeropen.com/counter/pdf/10.1186/s40537-020-00333-6.pdf)



In [8]:
branch.to_df()

,created_at,role,content,id,sender,recipient,metadata
0,2025-01-20 01:31:18.814877,system,{'system_message': ' You are a specialized res...,0e086733-565f-4d36-b465-d4dba66e7424,system,50112472-7615-4925-ae2f-73b26b3d2014,{'lion_class': 'lionagi.protocols.messages.sys...
1,2025-01-20 01:31:21.705987,user,"{'context': [], 'instruction': 'Write a compre...",ae79f33d-f255-4e6e-9eff-550aeb7492be,user,50112472-7615-4925-ae2f-73b26b3d2014,{'lion_class': 'lionagi.protocols.messages.ins...
2,2025-01-20 01:31:29.068366,assistant,"{'assistant_response': '```json { ""analysis""...",55496e88-918d-470d-855e-ad5dfb071dc8,50112472-7615-4925-ae2f-73b26b3d2014,user,{'model_response': {'id': 'gen-1737354682-iOLx...
3,2025-01-20 01:31:33.283213,action,"{'action_request': {'function': 'exa_search', ...",fcfbd037-127b-48af-8771-30a413bb5e90,50112472-7615-4925-ae2f-73b26b3d2014,29186376-96ee-441a-bfab-d5ea5bd6f758,{'lion_class': 'lionagi.protocols.messages.act...
4,2025-01-20 01:31:33.283303,action,{'action_request_id': 'fcfbd037-127b-48af-8771...,a0743416-32e5-43cd-8592-87c02d6cf496,29186376-96ee-441a-bfab-d5ea5bd6f758,50112472-7615-4925-ae2f-73b26b3d2014,{'lion_class': 'lionagi.protocols.messages.act...
5,2025-01-20 01:31:33.288458,user,{'context': [{'action_request_id': 'fcfbd037-1...,efb2ca9c-e194-44ef-9ecf-b145b9da9794,user,50112472-7615-4925-ae2f-73b26b3d2014,{'lion_class': 'lionagi.protocols.messages.ins...
6,2025-01-20 01:31:49.704805,assistant,"{'assistant_response': '```json { ""analysis""...",64ad479a-57b1-4574-958a-1fe4dc50b096,50112472-7615-4925-ae2f-73b26b3d2014,user,{'model_response': {'id': 'gen-1737354693-Mptk...
7,2025-01-20 01:31:49.705891,user,"{'context': [], 'instruction': 'given above re...",90ffc65f-357f-4d1b-ad29-8db25633fcd1,user,50112472-7615-4925-ae2f-73b26b3d2014,{'lion_class': 'lionagi.protocols.messages.ins...
8,2025-01-20 01:32:15.193196,assistant,"{'assistant_response': '```json { ""title"": ""...",005c235e-070b-44ed-b3c5-23848884eedc,50112472-7615-4925-ae2f-73b26b3d2014,user,{'model_response': {'id': 'gen-1737354710-mCzi...
